In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import timeit
import statistics
import time
import torch
import torch_geometric
import importlib

from data_utils import synthetic_data
from data_utils import graph_constructors
from data_utils import ant_graph_constructors

import networkx as nx

import importlib

In [15]:
importlib.reload(synthetic_data)
importlib.reload(ant_graph_constructors)

<module 'data_utils.ant_graph_constructors' from 'C:\\Users\\andy.knapper\\Documents\\OW\\Categorisation\\ML grouping\\GNN-for-trans-grouping\\data_utils\\ant_graph_constructors.py'>

In [16]:
data_list = [ant_graph_constructors.make_pyg_graph(*np.array(synthetic_data.make_a_group())).to(device) for _ in range(32)]

NameError: name 'device' is not defined

In [17]:
test_g = ant_graph_constructors.make_pyg_graph(*np.array(synthetic_data.make_a_group()))

In [18]:
test_g.edge_phero

tensor([[0.5055, 0.5611],
        [0.5302, 0.2238],
        [0.3237, 0.7709],
        [0.4761, 0.2963],
        [0.0846, 0.5442],
        [0.6393, 0.9605],
        [0.5831, 0.7731],
        [0.3591, 0.4472],
        [0.9548, 0.6468],
        [0.2735, 0.1339],
        [0.9069, 0.7644],
        [0.4462, 0.2640],
        [0.5380, 0.1946],
        [0.1367, 0.1988],
        [0.0761, 0.6853],
        [0.4409, 0.6016],
        [0.5720, 0.4933],
        [0.6593, 0.0785],
        [0.3445, 0.8214],
        [0.5725, 0.9906],
        [0.5365, 0.4399],
        [0.4587, 0.3981],
        [0.4624, 0.4189],
        [0.6114, 0.5835],
        [0.8775, 0.8521],
        [0.3846, 0.7332],
        [0.4846, 0.4821],
        [0.8178, 0.3394],
        [0.4764, 0.8778],
        [0.9720, 0.0578],
        [0.5831, 0.6233],
        [0.7807, 0.0671],
        [0.4825, 0.8753],
        [0.5415, 0.0157],
        [0.2819, 0.4836],
        [0.7610, 0.2236],
        [0.1160, 0.5961],
        [0.9576, 0.6897],
        [0.1

### Ideas

Since the ants brains won't be deep we'll spend more time deriving features than for a DNN.

In [38]:
#we'll replace the gnn edge feature function with one that calculates more features
#these need some more thought

from scipy.stats import rankdata
from itertools import combinations

def find_edge_features(inp_day_arr, inp_norm_amt_arr, inp_group_arr, edge):
    
    day_diff_feature_arr = np.zeros(7)
    edge_days_diff = int(inp_day_arr[edge[1]] - inp_day_arr[edge[0]])
    
    if edge_days_diff < 0:
        direction = -1
    elif edge_days_diff == 0:
        direction = 0
    else:
        direction = 1

    if np.abs(edge_days_diff) <= 5:
        day_diff_feature_arr[0] = 1
    elif np.abs(edge_days_diff) <= 9:
        day_diff_feature_arr[1] = 1
    elif np.abs(edge_days_diff) <= 12:
        day_diff_feature_arr[2] = 1
    elif np.abs(edge_days_diff) <= 16:
        day_diff_feature_arr[3] = 1
    elif np.abs(edge_days_diff) <= 24:
        day_diff_feature_arr[4] = 1
    elif np.abs(edge_days_diff) <= 36:
        day_diff_feature_arr[5] = 1
    else:
        day_diff_feature_arr[6] = 1
        
    #edge features
    edge_amt_diff_arr = np.array([inp_norm_amt_arr[edge[1]] - inp_norm_amt_arr[edge[0]]])

    mean_abs_amt_diff = np.mean(np.array([abs(x[0] - x[1]) for x in combinations(inp_norm_amt_arr, r=2)]))

    edge_amt_diff_to_mean_arr = np.abs(np.abs(edge_amt_diff_arr) - mean_abs_amt_diff)
    
    #below doesn't work 
    #edge_amt_rank_arr = (rankdata(edge_amt_diff_arr)-1) / len(edge_amt_diff_arr)

   
    return np.concatenate((edge_amt_diff_arr
                           , edge_amt_diff_to_mean_arr
                           #, edge_amt_rank_arr
                           , np.array([direction])
                           , day_diff_feature_arr), axis = None)


In [39]:
d_lst, a_lst, g_lst, t_lst = synthetic_data.make_a_group()
d_arr = np.array(d_lst)
a_arr = np.array(a_lst)
g_arr = np.array(g_lst)
t_arr = np.array(t_lst)
norm_a_arr = ant_graph_constructors.normalise_amounts(a_arr)

In [40]:
find_edge_features(d_arr, norm_a_arr, g_arr, [5,1])

array([ 0.47063406,  0.14930818, -1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ])